In [7]:
import math
import numpy as np
from numpy.typing import NDArray
import pandas as pd
import swifter
import pickle
from typing import Union, List, Any, Tuple
from qaoa_and_tdvp import QAOA, tdvp_optimize_qaoa, scipy_optimize, QAOAResult, Qobj
from MaxCut import MaxCut
from qutip import tensor, basis, Qobj, expect, variance

from benchmark import (
    get_all_connected,
    get_connected_rn_graph,
    get_rn_qubo,
    select_if_connected,
    Benchmark,
load_instances,
)
swifter;MaxCut;get_all_connected; get_connected_rn_graph; get_rn_qubo; select_if_connected; Benchmark; # noinspection PyStatementEffect

# convert data into DataFrame

In [8]:
with open('.//results/n4_results_itered.p', 'rb') as file:
    df = pickle.load(file)

df

instance  \
p i                                             
1 0  <MaxCut.MaxCut object at 0x7fc874219210>   
  1  <MaxCut.MaxCut object at 0x7fc8742182e0>   
  2  <MaxCut.MaxCut object at 0x7fc874219ea0>   
  3  <MaxCut.MaxCut object at 0x7fc874219cf0>   
  4  <MaxCut.MaxCut object at 0x7fc874219ba0>   
  5  <MaxCut.MaxCut object at 0x7fc874219b10>   
2 0  <MaxCut.MaxCut object at 0x7fc874219210>   
  1  <MaxCut.MaxCut object at 0x7fc8742182e0>   
  2  <MaxCut.MaxCut object at 0x7fc874219ea0>   
  3  <MaxCut.MaxCut object at 0x7fc874219cf0>   
  4  <MaxCut.MaxCut object at 0x7fc874219ba0>   
  5  <MaxCut.MaxCut object at 0x7fc874219b10>   
3 0  <MaxCut.MaxCut object at 0x7fc874219210>   
  1  <MaxCut.MaxCut object at 0x7fc8742182e0>   
  2  <MaxCut.MaxCut object at 0x7fc874219ea0>   
  3  <MaxCut.MaxCut object at 0x7fc874219cf0>   
  4  <MaxCut.MaxCut object at 0x7fc874219ba0>   
  5  <MaxCut.MaxCut object at 0x7fc874219b10>   
4 0  <MaxCut.MaxCut object at 0x7fc874219210>   
  1  <MaxCut.MaxCut object at 0x7fc8742182e0>   
  2  <MaxCut.MaxCut object at 0x7fc874219ea0>   
  3  <MaxCut.MaxCut object at 0x7fc874219cf0>   
  4  <MaxCut.MaxCut object at 0x7fc874219ba0>   
  5  <MaxCut.MaxCut object at 0x7fc874219b10>   
5 0  <MaxCut.MaxCut object at 0x7fc874219210>   
  1  <MaxCut.MaxCut object at 0x7fc8742182e0>   
  2  <MaxCut.MaxCut object at 0x7fc874219ea0>   
  3  <MaxCut.MaxCut object at 0x7fc874219cf0>   
  4  <MaxCut.MaxCut object at 0x7fc874219ba0>   
  5  <MaxCut.MaxCut object at 0x7fc874219b10>   

                                                  tdvp  \
p i                                                      
1 0  instance                  <MaxCut.MaxCut objec...   
  1  instance                  <MaxCut.MaxCut objec...   
  2  instance                  <MaxCut.MaxCut objec...   
  3  instance                  <MaxCut.MaxCut objec...   
  4  instance                  <MaxCut.MaxCut objec...   
  5  instance                  <MaxCut.MaxCut objec...   
2 0  instance                  <MaxCut.MaxCut objec...   
  1  instance                  <MaxCut.MaxCut objec...   
  2  instance                  <MaxCut.MaxCut objec...   
  3  instance                  <MaxCut.MaxCut objec...   
  4  instance                  <MaxCut.MaxCut objec...   
  5  instance                  <MaxCut.MaxCut objec...   
3 0  instance                  <MaxCut.MaxCut objec...   
  1  instance                  <MaxCut.MaxCut objec...   
  2  instance                  <MaxCut.MaxCut objec...   
  3  instance                  <MaxCut.MaxCut objec...   
  4  instance                  <MaxCut.MaxCut objec...   
  5  instance                  <MaxCut.MaxCut objec...   
4 0  instance                  <MaxCut.MaxCut objec...   
  1  instance                  <MaxCut.MaxCut objec...   
  2  instance                  <MaxCut.MaxCut objec...   
  3  instance                  <MaxCut.MaxCut objec...   
  4  instance                  <MaxCut.MaxCut objec...   
  5  instance                  <MaxCut.MaxCut objec...   
5 0  instance                  <MaxCut.MaxCut objec...   
  1  instance                  <MaxCut.MaxCut objec...   
  2  instance                  <MaxCut.MaxCut objec...   
  3  instance                  <MaxCut.MaxCut objec...   
  4  instance                  <MaxCut.MaxCut objec...   
  5  instance                  <MaxCut.MaxCut objec...   

                                                 scipy  \
p i                                                      
1 0  instance                  <MaxCut.MaxCut objec...   
  1  instance                  <MaxCut.MaxCut objec...   
  2  instance                  <MaxCut.MaxCut objec...   
  3  instance                  <MaxCut.MaxCut objec...   
  4  instance                  <MaxCut.MaxCut objec...   
  5  instance                  <MaxCut.MaxCut objec...   
2 0  instance                  <MaxCut.MaxCut objec...   
  1  instance                  <MaxCut.MaxCut objec...   
  2  instance                  

In [36]:
# scipy has no number set for 'steps' but we will use this field for computations
for p,i in df.index:
    df['scipy'].loc[p,i]['steps'] = df['scipy'].loc[p,i]['num_fun_calls']
    df['gradient_descent'].loc[p,i]['num_fun_calls'] = df['gradient_descent'].loc[p,i]['steps']
    for alg in {'scipy', 'tdvp', 'gradient_descent'}:
        res = df[alg].loc[p,i]
        try:
            res['success'] = res.res.success
        except  AttributeError:
            res['success'] = False
df['gradient_descent'].loc[1,0]

instance           NaN
p                  NaN
n                  NaN
delta_0            NaN
tollarance         NaN
algorithm          NaN
res                NaN
delta              NaN
value              NaN
path               NaN
steps              NaN
num_fun_calls      NaN
real duration      NaN
message            NaN
success          False
Name: gradient_descent, dtype: object

In [29]:
df['gradient_descent'].loc[1,0].res

nan

In [10]:
# with open('.//results//n4_p1.p','rb') as file:
#     df = pickle.load(file)

In [11]:
# with open("..//benchmarks//RK45_n10_p-1-5_Delta-1.p","rb") as file:
#     data: Benchmark = pickle.load(file)
#
# scipy_results = list()
# for d in data.results:
#     items = d.get("tdvp").__dict__.items()
#     for k,v in items:
#         d[k]=v
#         d["algorithm"] = "tdvp"
#     d["n"] = d["_qaoa"].n
#     dd = d.copy()
#     items = dd.get("scipy").__dict__.items()
#     for k,v in items:
#         dd[k]=v
#         dd["algorithm"] = "scipy"
#     dd["n"] = dd["_qaoa"].n
#     scipy_results.append(dd)
#
# data.results.extend(scipy_results)
#
# df = pd.DataFrame(data.results)
# df.pop("tdvp")
# df.pop("scipy")
# cols = df.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# df = df[cols]
#
# df['steps'] = df.apply(lambda row: int(row['_num_steps']) if row.algorithm=="tdvp" else row['_num_fun_calls'], axis=1)
#
# df.insert(2,"algorithm",df.pop("algorithm"))
# df.insert(3,"_optimal_fun_value",df.pop("_optimal_fun_value"))
# df.insert(4,'steps', df.pop('steps'))
# df.insert(6,"_prob",df.pop("_prob"))
#
#
# df_tdvp = df[df.algorithm.eq("tdvp")]
# df_scipy = df[df.algorithm.eq("scipy")]
#
# df.rename(columns={"_optimal_fun_value":"C", "_prob":"prob", '_duration':'duration', '_optimal_state':'final_state', '_optimal_parameters':'final_parameters', '_num_gates':'num_gates','_qaoa':'qaoa'}, inplace=True)

fix error when LinAlgerror exception occured and thererfore QAOAResult.duration = None leads to an error in `f"{self.duration:.2f}"`

In [12]:
# for (p,i) in df.index:
#     try:
#         df.loc[p,i].__repr__()
#     except TypeError:
#         print((p,i))
#         (p_er,i_er) = (p,i)
# df['tdvp'].loc[p_er,i_er].res.value = np.NAN
# df['tdvp'].loc[p_er,i_er].res.num_fun_calls = np.NAN
# df['tdvp'].loc[p_er,i_er].res.num_steps = np.NAN
# df['tdvp'].loc[p_er,i_er].res.parameters = list(np.NAN for _ in range(6))
# df['tdvp'].loc[p_er,i_er].res.optimizer_name = 'tdvp'
# df['tdvp'].loc[p_er,i_er].res.duration = np.NAN

# Compute performance measures

In [13]:
# dfp = df[df['p']==1]
# measures = ['expected approximation ratio', 'groundspace overlap', 'groundspace sharpness', 'Approximation ratio', 'circuit count', 'gate count']
# types = ['min','max', 'mean', 'stdv']
# measure_index=pd.MultiIndex.from_product([measures, types])
# summary = pd.DataFrame(columns=measure_index)
# summary

## basic functions

In [14]:
def prob(num:int, state:Qobj):
    n = state.shape[0]
    return abs(state.overlap(basis(n,num)))**2

def answer(state:Qobj)->int:
    return max(range(state.shape[0]), key=lambda x:prob(x,state))

def bin_answer(answer:int, n:int)->str:
    return format(answer,  f'0{n}b')

## approximation ratio

In [15]:
def best_cost_value(qaoa:QAOA):
    return qaoa.H.groundstate()[0]

def approximation_ratio(bin_answer:int, best_cost_value:float, qaoa:QAOA):
    i = list(int(b) for b in bin_answer)
    dims = [2 for _ in range(qaoa.n)]
    return expect(qaoa.H,basis(dims,i))/best_cost_value

def expectation_of_approximation_ratio(state:Qobj, best_cost_value:float, qaoa:QAOA):
    return (expect(qaoa.H, state))/best_cost_value


## time measures

In [16]:
path_example = df['tdvp'].loc[1,0].path

In [17]:
path_example

[(1.0, 1.0),
 (0.9308945581487198, 1.0484637612867636),
 (0.7871988325325365, 1.1663269277056116),
 (0.7827526471663281, 1.1878083790178537),
 (0.7880374293785674, 1.1947713393125265),
 (0.7962204175035178, 1.1983451340122915),
 (0.8004614859841208, 1.200017375920161)]

In [18]:
def num_metric_gates(n:int,p:int,qubo:NDArray):
    Q = np.count_nonzero(qubo)
    N = 2*p*(n+Q)
    C1 = 1/2 * n**4 + 2*n**3 + 7/2*n**2
    C2 = n**4 + 5/2*(n**3 + n**2)
    return N*C1*(7/3*p**2 +2*p + 1/3) + (2*C2-N*C1)*(1/2*p**2 +3/2*p)

def num_gradient_gates(n:int,p:int,qubo:NDArray):
    Q = np.count_nonzero(qubo)
    N = 2*p*(n+Q)
    return (n**4 -2*n**3 +n**2)/4 * (2*N+4) + 3*n*(n*(n-1))/2*(2*(N+3)) + 2*n**2*(2*N+3)

def num_qaoa_gates(n:int,p:int, qubo:NDArray):
    Q = np.count_nonzero(qubo)
    return 2*p*(n+Q)

def circuit_count(steps:int, algorithm:str, p:int, n:int):
    if algorithm =='tdvp':
        return steps*(p**2 + p)*(1/8*n**4 +1/2*n**3 -5/8*n**2) + 1
    if algorithm =="scipy":
        return steps + 1

def gate_count(steps: int, algorithm: str, p: int, n: int, qaoa: QAOA):
    if algorithm=='tdvp':
        return steps*(num_metric_gates(n,p,qaoa.qubo) + num_gradient_gates(n,p,qaoa.qubo)) + num_qaoa_gates(n,p,qaoa.qubo)
    if algorithm=='scipy':
        return (1+steps)*num_qaoa_gates(n,p,qaoa.qubo)

def path_length(path:List[Tuple[float]])->float:
    return sum(np.linalg.norm(np.array(path[i])-np.array(path[i+1])) for i in range(len(path)-1))

## Quality measures

In [19]:
# GROUNDSPACE IS NOW PART OF QAOA CLASS
# def groundspace(qaoa:QAOA):
#     eigenenergies, eigenstates = qaoa.H.eigenstates(sort='low')
#     groundenergy = eigenenergies[0]
#
#     num_of_eigenstates = 0
#     for e in eigenenergies:
#         if e==groundenergy:
#             num_of_eigenstates +=1
#         if e != groundenergy:
#             break
#     return eigenstates[:num_of_eigenstates]

def groundspace_overlap(groundstates:List[Qobj], state:Qobj):
    return  sum(abs(g.overlap(state))**2 for g in groundstates)

def groundspace_sharpness(groundstates:List[Qobj], state:Qobj):
    P_0 = sum(g*g.dag() for g in groundstates)
    return math.sqrt(variance(P_0,state))


## Compute everything

In [38]:
def compute_measures(res:pd.Series, qaoa:QAOA)->pd.Series:
    if res.success:
        state = qaoa.state(res.delta)
        res['answer'] = answer(state)
        res['bin_answer'] = bin_answer(res['answer'], res['n'])
        res['best_cost_value'] = best_cost_value(qaoa)
        res['approximation ratio'] = approximation_ratio(res['bin_answer'], res['best_cost_value'], qaoa)
        res['expectation approximation ratio']=expectation_of_approximation_ratio(state, res['best_cost_value'], qaoa)
        res['circuit count'] = circuit_count(res['steps'], res['algorithm'], res['p'], res['n'])
        res['gate count'] = gate_count(res['steps'], res['algorithm'], res['n'],res['p'], qaoa)
        res['path length'] = path_length(res['path'])
        res['groundspace overlap'] = groundspace_overlap(qaoa.groundspace, state)
        res['groundspace sharpness'] = groundspace_sharpness(qaoa.groundspace, state)
    else:
        res['answer'] = np.NaN
        res['bin_answer'] = np.NaN
        res['best_cost_value'] = np.NaN
        res['approximation ratio'] = np.NaN
        res['expectation approximation ratio']=np.NaN
        res['circuit count'] = np.NaN
        res['gate count'] = np.NaN
        res['path length'] = np.NaN
        res['groundspace overlap'] = np.NaN
        res['groundspace sharpness'] = np.NaN
    return res


In [27]:
def measures_mapping(row:pd.Series):
    qaoa = QAOA(row['instance'].qubo,p=row.name[0])
    if row.tdvp.algorithm is not np.NaN:
        row.tdvp = compute_measures(row.tdvp,qaoa)
    if row.scipy.algorithm is not np.NaN:
        row.scipy = compute_measures(row.scipy,qaoa)
    if row.gradient_descent.algorithm is not np.NaN:
        row.gradient_descent = compute_measures(row.gradient_descent,qaoa)
    # if row.scipy.algorithm is not None:
    #     row.scipy =compute_measures(row.scipy,qaoa)
    # if row.gradient_descent.algorithm is not None:
    #     row.gradient_descent =compute_measures(row.gradient_descent,qaoa)
    return row
def compute_measures_apply(df:pd.DataFrame)->pd.DataFrame:
    return df.swifter.apply(measures_mapping, axis=1)

In [39]:
df = compute_measures_apply(df)
df

Pandas Apply:   0%|          | 0/30 [00:00<?, ?it/s]

instance  \
p i                                             
1 0  <MaxCut.MaxCut object at 0x7fc874219210>   
  1  <MaxCut.MaxCut object at 0x7fc8742182e0>   
  2  <MaxCut.MaxCut object at 0x7fc874219ea0>   
  3  <MaxCut.MaxCut object at 0x7fc874219cf0>   
  4  <MaxCut.MaxCut object at 0x7fc874219ba0>   
  5  <MaxCut.MaxCut object at 0x7fc874219b10>   
2 0  <MaxCut.MaxCut object at 0x7fc874219210>   
  1  <MaxCut.MaxCut object at 0x7fc8742182e0>   
  2  <MaxCut.MaxCut object at 0x7fc874219ea0>   
  3  <MaxCut.MaxCut object at 0x7fc874219cf0>   
  4  <MaxCut.MaxCut object at 0x7fc874219ba0>   
  5  <MaxCut.MaxCut object at 0x7fc874219b10>   
3 0  <MaxCut.MaxCut object at 0x7fc874219210>   
  1  <MaxCut.MaxCut object at 0x7fc8742182e0>   
  2  <MaxCut.MaxCut object at 0x7fc874219ea0>   
  3  <MaxCut.MaxCut object at 0x7fc874219cf0>   
  4  <MaxCut.MaxCut object at 0x7fc874219ba0>   
  5  <MaxCut.MaxCut object at 0x7fc874219b10>   
4 0  <MaxCut.MaxCut object at 0x7fc874219210>   
  1  <MaxCut.MaxCut object at 0x7fc8742182e0>   
  2  <MaxCut.MaxCut object at 0x7fc874219ea0>   
  3  <MaxCut.MaxCut object at 0x7fc874219cf0>   
  4  <MaxCut.MaxCut object at 0x7fc874219ba0>   
  5  <MaxCut.MaxCut object at 0x7fc874219b10>   
5 0  <MaxCut.MaxCut object at 0x7fc874219210>   
  1  <MaxCut.MaxCut object at 0x7fc8742182e0>   
  2  <MaxCut.MaxCut object at 0x7fc874219ea0>   
  3  <MaxCut.MaxCut object at 0x7fc874219cf0>   
  4  <MaxCut.MaxCut object at 0x7fc874219ba0>   
  5  <MaxCut.MaxCut object at 0x7fc874219b10>   

                                                  tdvp  \
p i                                                      
1 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
2 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
3 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
4 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
5 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   

                                                 scipy  \
p i                                                      
1 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                                    <M...   
  3  instance                                    <M...   
  4  instance                                    <M...   
  5  instance                                    <M...   
2 0  instance                                    <M...   
  1  instance                                    <M...   
  2  instance                  

In [ ]:
df['tdvp'].loc[3,2]

# Compute the statistics

In [ ]:
# get one measure for all instances of one p and one algo
df.xs(3, level='p')['tdvp'].apply(lambda res: res['expectation approximation ratio'])

In [40]:
min_p = min(df.index, key= lambda x: x[0])[0]
max_p = max(df.index, key= lambda x: x[0])[0]

algs = ['tdvp', 'scipy', 'gradient_descent']
index = pd.MultiIndex.from_product([algs,range(min_p, max_p+1)], names=['algorithm', 'p'])

measures = ['expectation approximation ratio', 'groundspace overlap', 'groundspace sharpness', 'approximation ratio', 'circuit count', 'gate count', 'num_fun_calls', 'steps']
types = ['mean', 'std','min','max', ]
measure_index=pd.MultiIndex.from_product([measures, types])

stat = pd.DataFrame(index=index, columns=measure_index);stat

expectation approximation ratio                 \
                                              mean  std  min  max   
algorithm        p                                                  
tdvp             1                             NaN  NaN  NaN  NaN   
                 2                             NaN  NaN  NaN  NaN   
                 3                             NaN  NaN  NaN  NaN   
                 4                             NaN  NaN  NaN  NaN   
                 5                             NaN  NaN  NaN  NaN   
scipy            1                             NaN  NaN  NaN  NaN   
                 2                             NaN  NaN  NaN  NaN   
                 3                             NaN  NaN  NaN  NaN   
                 4                             NaN  NaN  NaN  NaN   
                 5                             NaN  NaN  NaN  NaN   
gradient_descent 1                             NaN  NaN  NaN  NaN   
                 2                             NaN  NaN  NaN  NaN   
                 3                             NaN  NaN  NaN  NaN   
                 4                             NaN  NaN  NaN  NaN   
                 5                             NaN  NaN  NaN  NaN   

                   groundspace overlap                groundspace sharpness  \
                                  mean  std  min  max                  mean   
algorithm        p                                                            
tdvp             1                 NaN  NaN  NaN  NaN                   NaN   
                 2                 NaN  NaN  NaN  NaN                   NaN   
                 3                 NaN  NaN  NaN  NaN                   NaN   
                 4                 NaN  NaN  NaN  NaN                   NaN   
                 5                 NaN  NaN  NaN  NaN                   NaN   
scipy            1                 NaN  NaN  NaN  NaN                   NaN   
                 2                 NaN  NaN  NaN  NaN                   NaN   
                 3                 NaN  NaN  NaN  NaN                   NaN   
                 4                 NaN  NaN  NaN  NaN                   NaN   
                 5                 NaN  NaN  NaN  NaN                   NaN   
gradient_descent 1                 NaN  NaN  NaN  NaN                   NaN   
                 2                 NaN  NaN  NaN  NaN                   NaN   
                 3                 NaN  NaN  NaN  NaN                   NaN   
                 4                 NaN  NaN  NaN  NaN                   NaN   
                 5                 NaN  NaN  NaN  NaN                   NaN   

                         ... gate count      num_fun_calls                 \
                    std  ...        min  max          mean  std  min  max   
algorithm        p       ...                                                
tdvp             1  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                 2  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                 3  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                 4  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                 5  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
scipy            1  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                 2  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                 3  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                 4  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                 5  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
gradient_descent 1  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                 2  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                 3  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                 4  NaN  ...        NaN  NaN           NaN  NaN  NaN  NaN   
                 5  NaN  ...        NaN  NaN

In [41]:
def compute_statistics(stat_row:pd.Series, data:pd.DataFrame):
    # LOOP THROUGH ALL STATISTICS
    algo, p = stat_row.name
    for meas, _ in stat_row.index:
        # get measure for all instances of one p and one algo
        data_meas:pd.Series = data.xs(p, level='p')[algo].apply(lambda res: res[meas])
        stat_row[meas,'mean'] = data_meas.mean()
        stat_row[meas,'std'] = data_meas.std()
        stat_row[meas,'min'] = data_meas.min()
        stat_row[meas,'max'] = data_meas.max()
    return stat_row

In [ ]:
compute_statistics(stat.loc['scipy',3], df)

## Save the processed data

In [44]:
stat = stat.swifter.apply(lambda x: compute_statistics(x, df), axis=1)
stat

Pandas Apply:   0%|          | 0/15 [00:00<?, ?it/s]

tdvp 1
tdvp 2
tdvp 3
tdvp 4
tdvp 5
scipy 1
scipy 2
scipy 3
scipy 4
scipy 5
gradient_descent 1
gradient_descent 2
gradient_descent 3
gradient_descent 4
gradient_descent 5


expectation approximation ratio                      \
                                              mean       std       min   
algorithm        p                                                       
tdvp             1                        0.555697  0.320149  0.133846   
                 2                        0.914538  0.023949  0.873142   
                 3                        0.926812  0.027653  0.895857   
                 4                        0.943432  0.049538  0.869999   
                 5                        0.969683  0.032257  0.935768   
scipy            1                        0.578786  0.215603  0.322760   
                 2                        0.850797  0.141596  0.562920   
                 3                        0.880351  0.073960  0.760516   
                 4                        0.832498  0.142717  0.623810   
                 5                        0.941948  0.074239  0.821507   
gradient_descent 1                             NaN       NaN       NaN   
                 2                             NaN       NaN       NaN   
                 3                             NaN       NaN       NaN   
                 4                             NaN       NaN       NaN   
                 5                             NaN       NaN       NaN   

                             groundspace overlap                      \
                         max                mean       std       min   
algorithm        p                                                     
tdvp             1  0.963225            0.464344  0.373130  0.019871   
                 2  0.947408            0.895209  0.049664  0.831052   
                 3  0.966206            0.866436  0.068250  0.795922   
                 4  0.974706            0.930693  0.062414  0.843536   
                 5  0.999978            0.960623  0.048394  0.906591   
scipy            1  0.795440            0.385312  0.334051  0.015630   
                 2  0.918540            0.786808  0.231791  0.335994   
                 3  0.973016            0.816939  0.087629  0.752143   
                 4  0.981445            0.681178  0.360276  0.043834   
                 5  0.995812            0.910507  0.117591  0.735778   
gradient_descent 1       NaN                 NaN       NaN       NaN   
                 2       NaN                 NaN       NaN       NaN   
                 3       NaN                 NaN       NaN       NaN   
                 4       NaN                 NaN       NaN       NaN   
                 5       NaN                 NaN       NaN       NaN   

                             groundspace sharpness            ...  \
                         max                  mean       std  ...   
algorithm        p                                            ...   
tdvp             1  0.962394              0.331865  0.164583  ...   
                 2  0.943499              0.297600  0.061860  ...   
                 3  0.967577              0.324697  0.090627  ...   
                 4  0.977556              0.232359  0.100608  ...   
                 5  0.999983              0.150136  0.143481  ...   
scipy            1  0.792068              0.346479  0.169024  ...   
                 2  0.946846              0.338920  0.098601  ...   
                 3  0.968308              0.366746  0.101873  ...   
                 4  0.980620              0.308058  0.130118  ...   
                 5  0.996740              0.214588  0.169398  ...   
gradient_descent 1       NaN                   NaN       NaN  ...   
                 2       NaN                   NaN       NaN  ...   
                 3       NaN                   NaN       NaN  ...   
                 4       NaN                   NaN       NaN  ...   
                 5       NaN                   NaN       NaN  ...   

                      gate count               num_fun_calls               \
                             min           max          mean         

In [46]:
if True:
    # noinspection PyUnreachableCode
    with open('.//results//n4_stats.p', 'wb') as file:
        pickle.dump(stat, file)

    with open('.//results//n4_data.p', 'wb') as file:
        pickle.dump(df, file)

# OLD

In [ ]:
# # df['groundspace'] = df.swifter.apply(lambda row: groundspace(row.qaoa), axis=1)
# # df.insert(3,'groundspace_overlap', df.swifter.apply(lambda row: groundspace_overlap(row['groundspace'], row['final_state']), axis=1))
# # df.insert(4, 'groundspace_sharpness', df.swifter.apply(lambda row: groundspace_sharpness(row['groundspace'], row['final_state']), axis=1))
# df['steps']
# df['circuit_count']= df.swifter.apply(lambda row: circuit_count(row['steps'], row['algorithm'], row['p'], row['n']), axis=1)
# df['gate_count'] =df.swifter.apply(lambda row: gate_count(row['steps'], row['algorithm'], row['p'], row['n'],
#                                                           row['qaoa']), axis=1)
# df['answer'] = df.swifter.apply(lambda row: answer(row['final_state']), axis=1)
# df['bin_answer'] = df.swifter.apply(lambda row: bin_answer(row['final_state']), axis=1)
# df.insert(3, 'best_cost_value', df.swifter.apply(lambda row: best_cost_value(row['qaoa']), axis=1))
# df.insert(3, 'approximation_ratio', df.swifter.apply(lambda row: approximation_ratio(row['bin_answer'],row['best_cost_value'],row['qaoa']), axis=1))
# df.insert(3, 'expectation_of_approximation_ratio', df.swifter.apply(lambda row: expectation_of_approximation_ratio(row['final_state'],row['best_cost_value'],row['qaoa']), axis=1))
# with open('..//benchmarks//n10_data.p', 'wb') as file:
#     pickle.dump(df,file)